### Create student csv file in the form of

|studentid|entranceyear|finalyear|majors|concentrations|finalmajor|finalconcentration|code-semester|Capstone
|--|--|--|--|--|--|--|--|--|
|00000000|2002|2006|[Undeclared, Engineering]|[Design]|Engineering|Design|['AHS0000 0203FA', ...]|ADE|

In [23]:
# importing original dataset to work on 
import pandas as pd
data = pd.read_csv('../data/registrations.csv') 

In [24]:
# overview of original dataframe
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34732 entries, 0 to 34731
Data columns (total 14 columns):
studentid              34732 non-null int64
semester               34732 non-null object
grade                  34732 non-null object
major                  34732 non-null object
concentration          13268 non-null object
code                   34732 non-null object
section                34732 non-null object
coursetitle            34732 non-null object
sectiontitle           4930 non-null object
professor              32476 non-null object
course-sectiontitle    34732 non-null object
code-semester          34732 non-null object
Unnamed: 0             34732 non-null int64
coursetype             15902 non-null object
dtypes: int64(2), object(12)
memory usage: 4.0+ MB


In [3]:
# Creating the major column for each student
import string

major_fullnames = []
for index, registration in data.iterrows():
    major = string.strip(str(registration.major))
    
    concentration = ""
    if type(registration.concentration) == str:
        concentration = string.strip(str(registration.concentration))
        
    major_fullname = string.strip(major + " " + concentration)
    major_fullnames.append(major_fullname)
    
data['major_fullname'] = major_fullnames

In [4]:
# Creating columns for entry year and leave year for students (or last recorded year)
def semester_year(semester):
    fs = semester[4:6]
    if fs == 'FA':
        year = semester[:2]
    elif fs == 'SP':
        year = semester[2:4]
    return int(year)

data['term'] = data['semester'].str[4:6]
data['year'] = data['semester'].apply(semester_year)

In [5]:
# Applying the manual course categorization from 'course_type.ipynb' so we observe the trend of each student

def semester_year(year):
    semyear = 2000 + int(year)
    return semyear

def me(coursetype):
    if coursetype == 'me':
        return 1
    else:
        return 0

def ece(coursetype):
    if coursetype == 'ece':
        return 1
    else:
        return 0
    
def cs(coursetype):
    if coursetype == 'cs':
        return 1
    else:
        return 0
    
def design(coursetype):
    if coursetype == 'design':
        return 1
    else:
        return 0

def bio(coursetype):
    if coursetype == 'bio':
        return 1
    else:
        return 0

data['semester_year'] = data['year'].apply(semester_year)
data['me'] = data['coursetype'].apply(me)
data['ece'] = data['coursetype'].apply(ece)
data['cs'] = data['coursetype'].apply(cs)
data['design'] = data['coursetype'].apply(design)
data['bio'] = data['coursetype'].apply(bio)

In [29]:
studentd = {'studentid':[],
            'entranceyear':[],
            'finalyear':[],
            'majors': [], 
            'concentrations': [],
            'major-fullnames': [],
            'code':[],
            'code-semester':[], 
            'me':[],'ece':[],'cs':[],'design':[],'bio':[]}

grouped = data.groupby('studentid')
for group in grouped:
    studentd['studentid'].append(group[0])
    studentd['code'].append(group[1]['code'].unique())
    studentd['code-semester'].append(group[1]['code-semester'].unique())
    studentd['majors'].append(group[1]['major'].unique())
    studentd['concentrations'].append(group[1]['concentration'].unique())
    studentd['major-fullnames'].append(group[1]['major_fullname'].unique())
    studentd['entranceyear'].append(min(group[1]['semester_year'].unique()))
    studentd['finalyear'].append(max(group[1]['semester_year'].unique()))
    studentd['me'].append(group[1]['me'].sum())
    studentd['ece'].append(group[1]['ece'].sum())
    studentd['cs'].append(group[1]['cs'].sum())
    studentd['design'].append(group[1]['design'].sum())
    studentd['bio'].append(group[1]['bio'].sum())


In [8]:
# creating stu dataframe
student = pd.DataFrame(studentd)

In [9]:
def last_element(element):
    return element[-1]

def capstone(codes):
    if 'ENGR4199' in codes:
        return 'ADE'
    elif 'ENGR4290' in codes:
        return 'ADE'
    elif 'ENGR4190' in codes:
        return 'SCOPE'
    else:
        return

student['capstone'] = student['code'].apply(capstone)
student['finalmajor'] = student['majors'].apply(last_element)
student['finalconcentration'] = student['concentrations'].apply(last_element)

In [10]:
# Deciding majors on the main major and submajor, if student doesnt have submajor, will return nan
import numpy as np
for i in student.index:
    if student.finalmajor[i][0:11] == 'Engineering':
        pass
    else:
        student.finalconcentration[i] = np.nan

/home/tj/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
def ignan(conc):
    if type(conc) == type(np.nan):
        return ''
    else:
        return ' '+conc
    
student['finalmajor_full'] = student['finalmajor']+student['finalconcentration'].apply(ignan)

In [12]:
# Defining columns for final dataframe
columns = ['studentid','entranceyear','finalyear',
           'majors','concentrations','finalmajor','finalconcentration','finalmajor_full',
           'code-semester','me','ece','design','cs','bio']
student = student[columns]

In [13]:
student.to_csv('../data/student.csv', index = False)
student.head(10)

,studentid,entranceyear,finalyear,capstone,majors,concentrations,finalmajor,finalconcentration,finalmajor_full,code-semester,me,ece,design,cs,bio
0,10136845,2013,2016,None,"[Undeclared, Mechanical Engineering]",[nan],Mechanical Engineering,NaN,Mechanical Engineering,"[AHSE1145 1314FA, AHSE2141 1314SP, AHS...",6,1,4,3,1
1,10191843,2014,2015,None,[Undeclared],[nan],Undeclared,NaN,Undeclared,"[AHSE1145 1415FA, AHSE1515 1415SP, ENG...",3,1,2,1,0
2,10208842,2014,2016,None,"[Undeclared, Engineering]",[Computing],Engineering,Computing,Engineering Computing,"[AHSE1199 1415FA, AHSE1515 1415SP, ENG...",0,2,2,3,0
3,10324389,2007,2011,ADE,"[Undeclared, Engineering]",[Self Designed],Engineering,Self Designed,Engineering Self Designed,"[AHSE1199 0708FA, AHSE1500 0809FA, AHS...",2,2,6,1,3
4,10463379,2003,2007,SCOPE,"[Undeclared, Electr'l & Computer Engr, Mechani...",[nan],Mechanical Engineering,NaN,Mechanical Engineering,"[AHS1101 0304FA, AHS1111 0304SP, AHS...",7,2,3,1,0
5,10478316,2011,2015,SCOPE,"[Undeclared, Mechanical Engineering]",[nan],Mechanical Engineering,NaN,Mechanical Engineering,"[AHSE1100 1112FA, AHSE1500 1314SP, AHS...",9,2,3,0,0
6,10510869,2008,2012,SCOPE,"[Undeclared, Engineering]",[Bioengineering],Engineering,Bioengineering,Engineering Bioengineering,"[AHSE1100 0809FA, AHSE1500 1011SP, AHS...",1,3,3,1,3
7,10543230,2009,2013,ADE,"[Undeclared, Mechanical Engineering]",[nan],Mechanical Engineering,NaN,Mechanical Engineering,"[AHSE1199 0910FA, AHSE1500 1011FA, AHS...",6,1,5,1,0
8,10585887,2004,2008,SCOPE,"[Undeclared, Engineering]",[Bioengineering],Engineering,Bioengineering,Engineering Bioengineering,"[AHSE1101 0405FA, AHSE1500 0405SP, AHS...",1,2,3,1,5
9,10784794,2011,2015,SCOPE,"[Undeclared, Engineering]",[Design],Engineering,Design,Engineering Design,"[AHSE1155 1112SP, AHSE1500 1314FA, AHS...",0,2,5,6,0
